# <font color='#7ad7f0'><font size='30'> WEB SCRAPING PART</font></font>
#### Scraping the-numbers.com for movie data


In [2]:
#Importing necessary libraries

from bs4 import BeautifulSoup 
import urllib
import requests
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import re

## Defining additional functions


In [94]:
#functions for populating feature lists

def movdet(cast): #gets actor and director names
    urlmain = 'https://www.the-numbers.com'
    urlmov=urlmain+cast
    #Request HTML and parse
    response = requests.get(urlmov)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    actor= list()
    try:
        for i in soup.find_all(itemprop = "actor"):
            for a in i.find_all('a', href=True):
                actor.append(a['href'])
        actor2= list()
        for i in range(len(actor)):
            a, b= str(actor[i]).split("-", 1)
            actor2.append(b)
        star.append(actor2[0])
        director1= list()
        for i in soup.find_all(itemprop = "director"):
            for a in i.find_all('a', href=True):
                director1.append(a['href'])
        director2= list()
        for i in range(len(director1)):
            a, b= str(director1[i]).split("-", 1)
            director2.append(b)
        director.append(director2[0])
    except:
        star.append("None")
        director.append("None")
    

def movsum(sumlist): # gets mpaa rating and genre
    tempurl="https://www.the-numbers.com"+str(sumlist)
    tempres=requests.get(tempurl)
    temppage = tempres.text
    tempsoup = BeautifulSoup(temppage,"lxml")

    mpaa=tempsoup.find("a", href=re.compile("market/mpaa-rating"))
    mpaa= str(mpaa).strip('<a href="/market/mpaa-rating/').split("-(US)", 1)
    rating.append(mpaa[0])

    gen=tempsoup.find("a", href=re.compile("/market/genre/"))
    gen= str(gen).strip('<a href="/market/genre/').split('">', 1)
    genre.append(gen[0])


## Scraping names of films
this list will be used to later used to scrape page of each film individually 

In [92]:
#getting inital data from html

url1 = 'https://www.the-numbers.com/movie/budgets/all'


response = requests.get(url1)
response.status_code 
myhtml= response.text

soup = BeautifulSoup(myhtml, 'lxml')




In [93]:
#lists for  features i actually was able to scrape
rank=[]
budget=[]
dom_gross=[]
ww_gross=[]
star=list()
costar=list()
director=list()
producer=list()
rating=list()
genre=list()



## Data cleaning

In [95]:
#cleaning data to retain only numeric values
budgets=[]
for elem in soup.find_all(class_='data'):
    budgets.append(elem.text)

for i in range(len(budgets)):
    budgets[i]=(re.sub("[^0-9]", "",  budgets[i]))
budgets= list(budgets)

 # rank number 
for i in range(0, len(budgets), 4):
    rank.append(budgets[i])

# budget
for i in range(1, len(budgets), 4):
    budget.append(budgets[i])

# domestic gross
for i in range(2, len(budgets), 4):
    dom_gross.append(budgets[i])

 # worldwide gross
for i in range(3, len(budgets), 4):
    ww_gross.append(budgets[i])



In [85]:
#lists for storing links to each film 
link_lst= list() #temp list of all links in table
sumlist=list() #list of links to movie summary tab
cast = list()  #list of links to movie cast&crew tab

table = soup.find('table')
 
for a in table.find_all('a', href=True):
    link_lst.append(a['href'])

dates= list()
for i in range(0, len(link_lst), 2):
    link_lst[i]= link_lst[i].replace('/box-office-chart/daily/', '')
    dates.append(link_lst[i])
    

for i in range(1, len(link_lst), 2):
    sumlist.append(link_lst[i])
    link_lst[i]= link_lst[i].replace('=summary', '=cast-and-crew')
    cast.append(link_lst[i])






In [96]:
#scraping each mov page 
for i in range(len(cast)):
    movdet(cast[i])
for j in range(len(sumlist)):
    movsum(sumlist[i])

In [97]:
# verifying all my lists have same number of elements
print('cast={}'.format(len(cast)))
print('sumlist={}'.format(len(sumlist)))
print('dir={}'.format(len(director)))
print('star={}'.format(len(star)))
print('gen={}'.format(len(genre)))
print('rat={}'.format(len(rating)))

cast=100
sumlist=100
dir=100
star=100
gen=100
rat=100


# Creating dataframe from feature lists

In [98]:
#creates pandas df from all lists
df=pd.DataFrame(list(zip(rank, budget, dom_gross, ww_gross, director, star, dates, genre, rating)), 
               columns =['Number', 'Budget', 'Domestic Gross', 'Worldwide Gross', 'Director', 'Main Actor', 'Release Date', 'Genre', 'MPAA'])
print(df.head)

<bound method NDFrame.head of    Number     Budget Domestic Gross Worldwide Gross               Director  \
0       1  400000000      858373000      2797800564         Andy-Wachowski   
1       2  379000000      241063875      1045663875  Christopher-McQuarrie   
2       3  365000000      459005868      1396099202           Bryan-Singer   
3       4  306000000      936662225      2065478084         Jon-Turteltaub   
4       5  300000000      678815482      2048359754             Doug-Liman   
..    ...        ...            ...             ...                    ...   
95     96  180000000      223808164       532508025      Wolfgang-Petersen   
96     97  180000000      140125968       256585882      M-Night-Shyamalan   
97     98  180000000      126643061       348902025          Steve-Hickner   
98     99  180000000       73864507       180047784         Hoyt-H-Yeatman   
99    100  180000000       40479370       215098356         Chris-Williams   

        Main Actor Release Date  

In [99]:
#makes csv file from df

df.to_csv(r'/Users/ramonmartin/Documents/METIS/Project 2/movdf.csv', index= False)